In [1]:
import json
import re
import unicodedata
import os
from vncorenlp import VnCoreNLP

subject_files = {
    "văn": "output/văn.json",
    "sử": "output/sử.json",
    "địa": "output/địa.json",
    "anh": "output/anh_văn.json"
}

all_data = {}

for subject, file_path in subject_files.items():
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            all_data[subject] = json.load(f)
        print(f"Loaded {len(all_data[subject])} questions from {subject}")
    except FileNotFoundError:
        print(f"Warning: File not found - {file_path}")
        continue



Loaded 3239 questions from văn
Loaded 670 questions from sử
Loaded 838 questions from địa
Loaded 72 questions from anh


In [2]:
def simple_tokenize(text):
    """
    Simple Vietnamese tokenization function
    Args:
        text (str): Input text to tokenize
    Returns:
        str: Tokenized text with spaces between words
    """
    if not isinstance(text, str):
        return ""
    
    # Tách từ đơn giản bằng khoảng trắng
    words = text.split()
    
    # Tách số và chữ
    processed_words = []
    for word in words:
        # Tách số và chữ
        word = re.sub(r'([A-Za-z])([0-9])', r'\1 \2', word)
        word = re.sub(r'([0-9])([A-Za-z])', r'\1 \2', word)
        # Tách chữ hoa
        word = re.sub(r'([a-z])([A-Z])', r'\1 \2', word)
        processed_words.append(word)
    
    return ' '.join(processed_words)

In [3]:
def clean_text(text):
    if not isinstance(text, str):
        return ""
    # Chuẩn hóa Unicode (xử lý dấu tiếng Việt)
    text = unicodedata.normalize("NFC", text)

    # Bỏ xuống dòng, tab, khoảng trắng thừa
    text = re.sub(r"\s+", " ", text)

    # Bỏ ký tự đặc biệt không cần thiết
    text = re.sub(r"[^\w\sÀ-ỹ]", "", text)

    return text.strip()

In [4]:
# Lấy và làm sạch câu hỏi
qa_pairs = []
for subject, data in all_data.items():
    for item in data:
        if "question" in item and "answers" in item:
            clean_q = clean_text(item["question"])
            clean_ans = [clean_text(ans) for ans in item["answers"]]
            correct_ans = item.get("correct_answer")
            explanation = clean_text(item.get("explanation", ""))
            
            if clean_q and clean_ans:  # chỉ lấy nếu có cả câu hỏi và câu trả lời
                qa_pairs.append({
                    "subject": subject,
                    "question": clean_q,
                    "answers": clean_ans,
                    "correct_answer": correct_ans,
                    "explanation": explanation if explanation else None
                })

In [5]:
# Tách từ
processed_qa = []
for qa in qa_pairs:
    processed_qa.append({
        "subject": qa["subject"],
        "question": simple_tokenize(qa["question"]),
        "answers": [simple_tokenize(ans) for ans in qa["answers"]],
        "correct_answer": qa["correct_answer"],
        "explanation": simple_tokenize(qa["explanation"]) if qa["explanation"] else None
    })

In [6]:
# Lưu kết quả vào file
if not os.path.exists("data_processed"):
    os.makedirs("data_processed")

questions_by_subject = {}
for qa in processed_qa:
    subject = qa['subject']
    if subject not in questions_by_subject:
        questions_by_subject[subject] = []
    questions_by_subject[subject].append(qa)

for subject, questions in questions_by_subject.items():
    output_file = f"data_processed/{subject}_processed.txt"
    
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(f"Tổng số câu hỏi môn {subject}: {len(questions)}\n\n")
        
        for i, qa in enumerate(questions, 1):
            f.write(f"Câu {i}:\n")
            f.write(f"Câu hỏi: {qa['question']}\n")
            f.write("Các đáp án:\n")
            for j, ans in enumerate(qa['answers'], 1):
                f.write(f"  {j}. {ans}\n")
            f.write(f"Đáp án đúng: {qa['correct_answer']}\n")
            if qa['explanation']:
                f.write(f"Giải thích: {qa['explanation']}\n")
            f.write("\n" + "-"*50 + "\n\n")
    
    print(f"\nĐã xử lý và lưu {len(questions)} câu hỏi môn {subject} vào file '{output_file}'")


Đã xử lý và lưu 3210 câu hỏi môn văn vào file 'data_processed/văn_processed.txt'

Đã xử lý và lưu 670 câu hỏi môn sử vào file 'data_processed/sử_processed.txt'

Đã xử lý và lưu 834 câu hỏi môn địa vào file 'data_processed/địa_processed.txt'

Đã xử lý và lưu 35 câu hỏi môn anh vào file 'data_processed/anh_processed.txt'
